# Improvement Trial With Smaller Temperatures, Smaller Discounts

starting initial tests with a smaller number of trials just to see general performance

In [1]:
import gpu_rec
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 0.01

init_game = Game(n_players=5, q_lag=1, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []



In [3]:
temperatures = [ 0.2, 0.1, 0.1, 0.05, 0.01, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001, 0.001, 0., 0., 0.]
n_trials_per_cycle = 80
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print '---TEMPERATURE: %.03f---' % temperature
    print '+IDX=%d+' % temp_index
    if temp_index==9:
        test_config['discount'] = 0.05
    elif temp_index==10:
        test_config['discount'] = 0.1
    elif temp_index==11:
        test_config['discount'] = 0.2
    elif temp_index==12:
        test_config['discount'] = 0.5
    elif temp_index==13:
        test_config['discount'] = 1
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

---TEMPERATURE: 0.200---
+IDX=0+
in cycle 0
................................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 2.6
NUMBER OF TURNS AVERAGE: 266.2
---PLAYER 0 STATS---
WIN AVERAGE: 0.196
SCORE AVERAGE: -21.8
---PLAYER 1 STATS---
WIN AVERAGE: 0.050
SCORE AVERAGE: -49.6
---PLAYER 2 STATS---
WIN AVERAGE: 0.483
SCORE AVERAGE: -11.8
---PLAYER 3 STATS---
WIN AVERAGE: 0.204
SCORE AVERAGE: -21.7
---PLAYER 4 STATS---
WIN AVERAGE: 0.071
SCORE AVERAGE: -30.7
---TEMPERATURE: 0.100---
+IDX=1+
in cycle 0
................................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 2.

ai.py:407: RuntimeWarning: invalid value encountered in divide
  return (selected)/np.sum(selected)
ai.py:379: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


...............................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 3.9
NUMBER OF TURNS AVERAGE: 241.7
---PLAYER 0 STATS---
WIN AVERAGE: 0.013
SCORE AVERAGE: -38.2
---PLAYER 1 STATS---
WIN AVERAGE: 0.125
SCORE AVERAGE: -28.3
---PLAYER 2 STATS---
WIN AVERAGE: 0.446
SCORE AVERAGE: -11.1
---PLAYER 3 STATS---
WIN AVERAGE: 0.225
SCORE AVERAGE: -25.0
---PLAYER 4 STATS---
WIN AVERAGE: 0.221
SCORE AVERAGE: -22.1
---TEMPERATURE: 0.010---
+IDX=5+
in cycle 0
................................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 4.4
NUMBER OF TURNS AVERAGE: 242.5
---PLAYER 0 

its seems that temperature=0 produces the least desirable results, and increasing the discount factor doesn't help (since it's scaled to [0,1], anyway...)...also, why is player 0's score improving at all in spite of the fact that no training is done for that player???

In [4]:
print game.turn
print i
print cycle
print temp_index
for player in players:
    print player
    print len(player.determine_possible_tracks())

275
4
2
14
player with 0/7 completed/uncompleted tickets, 4 train cards, 0 cars, and 
		-22 total points
0
player with 0/6 completed/uncompleted tickets, 67 train cards, 16 cars, and 
		9 total points
71
player with 0/7 completed/uncompleted tickets, 27 train cards, 18 cars, and 
		-15 total points
76
player with 1/6 completed/uncompleted tickets, 4 train cards, 6 cars, and 
		-33 total points
35
player with 0/7 completed/uncompleted tickets, 0 train cards, 7 cars, and 
		-31 total points
0


In [5]:
np.mean(overall_winning_scores,1)

array([  2.57083333,   2.37916667,   2.0375    ,   1.2875    ,
         3.89166667,   4.39166667,   4.1       ,   3.35      ,
         7.17083333,   7.0875    ,   9.        ,   9.05833333,
         9.3       ,   7.86666667,  10.59583333])

In [6]:
np.mean(overall_player_scores, ( 2,))

array([[-21.7625    , -49.62916667, -11.78333333, -21.69166667,
        -30.70416667],
       [-23.33333333, -45.40416667, -12.95833333, -20.75416667, -31.3625    ],
       [-25.03333333, -46.67916667, -13.29583333, -21.49166667,
        -29.48333333],
       [-21.4375    , -41.91666667, -12.42083333, -25.55      ,
        -28.98333333],
       [-38.19583333, -28.27916667, -11.07083333, -24.9625    , -22.125     ],
       [-37.49583333, -24.7375    , -12.76666667, -24.42083333,
        -22.10833333],
       [-39.29583333, -25.3875    , -14.21666667, -23.        , -21.15      ],
       [-37.70416667, -25.6625    , -13.2375    , -23.41666667, -22.8       ],
       [-23.325     , -26.64583333,  -5.89583333, -21.575     , -27.925     ],
       [-35.17916667, -22.42916667,  -7.2375    , -25.54583333,
        -21.60833333],
       [-41.35416667, -18.975     ,  -6.97916667, -26.31666667,
        -19.22916667],
       [-40.19583333, -21.04583333,  -4.38333333, -26.45      ,
        -21.8041666

In [9]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test00_%%s.h5' % i)

This training methodology seems to be improving. Perhaps next time I should transfer the best performing AIs to the two worst-performing ones (excluding player 0)